In [ ]:
from sentence_transformers import SentenceTransformer

import numpy as np

model = SentenceTransformer("./stsb-distilbert-base-quora-duplicate-questions")

c:\Users\99896\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


torch.Size([4, 4])


In [ ]:

sentences = [
   "How does chocolate get made?",
    "What is the process of making chocolate?",
    "How do you prepare for exams effectively?",
    "What are some effective study methods for exams?",
    "What is the best way to learn Python?",
    "How can I start learning Python?",
    "What are the steps involved in machine learning?",
    "Explain the process of machine learning.",
    "How do I make pasta?",
    "Can you tell me how pasta is made?",
    "What is the best book for AFCAT preparation?",
    "Can you suggest some books for AFCAT?",
    "How does a car engine work?",
    "Explain the working principle of a car engine.",
    "How to improve public speaking skills?",
    "What are some tips for improving public speaking?",
    "What is the capital of France?",
    "Where is the Eiffel Tower located?",
    "What is the time now?",
    "How to bake a cake?",
    "Can you give me a cake recipe?",
    "How does a blockchain work?",
    "Explain blockchain technology.",
    "What is artificial intelligence?",
    "Define artificial intelligence in simple terms."
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities)

# Convert the similarities to binary similarities
similarities=similarities.numpy()
printed=[]
binary_similarities = (similarities > 0.75).astype(int)
for i in range(len(similarities)):
    for j in range(len(similarities)):
        if i != j and binary_similarities[i][j] ==1 and i not in printed:
            printed.append(i)
            printed.append(j)
            print(" 1:", sentences[i])
for i in range(len(similarities)):
    if i not in printed:
        print(" 2:", sentences[i])

print(binary_similarities)

tensor([[ 1.0000e+00,  8.7658e-01, -4.5598e-02, -8.0722e-02,  1.1243e-01,
          1.9686e-01,  1.1571e-01,  1.4253e-01,  2.3099e-01,  2.2219e-01,
         -5.9828e-02, -8.9349e-02,  8.4794e-02,  7.2442e-02, -2.6264e-02,
         -1.2309e-02, -1.4192e-02,  1.7120e-02, -1.0189e-01,  5.9221e-01,
          5.3040e-01,  8.9872e-02,  6.8932e-02,  1.7726e-01,  1.6068e-01],
        [ 8.7658e-01,  1.0000e+00,  3.6747e-02,  2.3680e-02,  1.4728e-01,
          2.2269e-01,  2.0482e-01,  2.1311e-01,  2.8756e-01,  2.4903e-01,
         -2.4927e-02, -6.5546e-02,  9.5585e-02,  1.3597e-01, -8.3534e-03,
          1.0951e-02,  2.1105e-03,  1.4016e-03, -8.6109e-02,  5.6185e-01,
          4.8739e-01,  6.5596e-02,  7.8036e-02,  1.3842e-01,  1.3997e-01],
        [-4.5598e-02,  3.6747e-02,  1.0000e+00,  6.1158e-01,  1.9979e-01,
          1.7288e-01,  1.8189e-01,  1.2918e-01,  1.5837e-01,  1.1906e-01,
          3.6659e-01,  2.6083e-01,  2.6515e-02, -4.5575e-02,  2.1987e-01,
          2.1809e-01,  3.9817e-02, -

In [31]:

from sklearn.metrics import accuracy_score
true_labels = [
    [0, 1],  # "How does chocolate get made?" and "What is the process of making chocolate?"
    [2, 3],  # Exam preparation questions
    [4, 5],  # Python learning questions
    [6, 7],  # Machine learning questions
    [8, 9],  # Pasta making questions
    [10, 11], # AFCAT book suggestions
    [12, 13], # Car engine questions
    [14, 15], # Public speaking questions
    [16, 17], # France and Eiffel Tower
    [18],     # "What is the time now?" is unique
    [19, 20], # Cake recipe questions
    [21, 22], # Blockchain questions
    [23, 24]  # AI questions
]

# Flatten the true_labels to create a list of true binary relations (True or False)
true_binary = np.zeros_like(binary_similarities, dtype=int)
for cluster in true_labels:
    for i in cluster:
        for j in cluster:
            if i != j:
                true_binary[i][j] = 1

# Calculate the accuracy by comparing the binary similarity matrix with the ground truth
accuracy = accuracy_score(true_binary.flatten(), binary_similarities.flatten())

print("Accuracy:", accuracy)

Accuracy: 0.9536


In [36]:

from sklearn.cluster import AgglomerativeClustering

sentences = [
    "How does chocolate get made?",
    "What is the process of making chocolate?",
    "How do you prepare for exams effectively?",
    "What are some effective study methods for exams?",
    "What is the best way to learn Python?",
    "How can I start learning Python?",
    "What are the steps involved in machine learning?",
    "Explain the process of machine learning.",
    "How do I make pasta?",
    "Can you tell me how pasta is made?",
    "What is the best book for AFCAT preparation?",
    "Can you suggest some books for AFCAT?",
    "How does a car engine work?",
    "Explain the working principle of a car engine.",
    "How to improve public speaking skills?",
    "What are some tips for improving public speaking?",
    "What is the capital of France?",
    "Where is the Eiffel Tower located?",
    "What is the time now?",
    "How to bake a cake?",
    "Can you give me a cake recipe?",
    "How does a blockchain work?",
    "Explain blockchain technology.",
    "What is artificial intelligence?",
    "Define artificial intelligence in simple terms."
]

embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
similarities=similarities.numpy()
clustering = AgglomerativeClustering(n_clusters=None, metric='precomputed', linkage='average', distance_threshold=0.25)
labels = clustering.fit_predict(1 - similarities)
print(labels)
# Step 5: Find the most representative question for each cluster
unique_questions = []

for label in np.unique(labels):
    cluster_indices = np.where(labels == label)[0]
    
    # Step 5.1: Calculate centroid of the cluster
    cluster_embeddings = embeddings[cluster_indices]
    centroid = np.mean(cluster_embeddings, axis=0)
    
    # Step 5.2: Find the question closest to the centroid
    similarities_to_centroid = cosine_similarity([centroid], cluster_embeddings)[0]
    closest_question_idx = np.argmax(similarities_to_centroid)
    unique_questions.append(sentences[cluster_indices[closest_question_idx]])

[ 6  6 14  9  8  8  4  4  5  5  1  1  2  2 11 11 12 10 13  0  0  7  7  3
  3]


In [34]:
print(f"Original number of questions: {len(sentences)}")
print(f"Number of unique questions after clustering: {len(unique_questions)}")
print("Unique questions:")
print(unique_questions)

Original number of questions: 25
Number of unique questions after clustering: 15
Unique questions:
['How to bake a cake?', 'What is the best book for AFCAT preparation?', 'How does a car engine work?', 'What is artificial intelligence?', 'Explain the process of machine learning.', 'How do I make pasta?', 'What is the process of making chocolate?', 'How does a blockchain work?', 'How can I start learning Python?', 'What are some effective study methods for exams?', 'Where is the Eiffel Tower located?', 'How to improve public speaking skills?', 'What is the capital of France?', 'What is the time now?', 'How do you prepare for exams effectively?']
